In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO

In [30]:
# goal here is to combine everything into one csv file so its nice and ready to be used
mvps = pd.read_csv("mvpsfix.csv")
mvps.head()

,Unnamed: 0,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,0,1,Magic Johnson,29,LAL,42.5,664.5,850,0.782,77,...,7.9,12.8,1.8,0.3,0.509,0.314,0.911,16.1,0.267,1989
1,1,2,Michael Jordan,25,CHI,27.5,598.8,850,0.704,81,...,8.0,8.0,2.9,0.8,0.538,0.276,0.850,19.8,0.292,1989
2,2,3,Karl Malone,25,UTA,5.0,362.0,850,0.426,80,...,10.7,2.7,1.8,0.9,0.519,0.313,0.766,15.2,0.233,1989
3,3,4,Patrick Ewing,26,NYK,8.0,200.0,850,0.235,80,...,9.3,2.4,1.5,3.5,0.567,0.000,0.746,10.9,0.181,1989
4,4,5,Hakeem Olajuwon,26,HOU,2.0,179.3,850,0.211,82,...,13.5,1.8,2.6,3.4,0.508,0.000,0.696,12.4,0.197,1989


In [31]:
mvps = mvps[["Player", "Year", "Pts Won", "Share"]]
mvps.head()

,Player,Year,Pts Won,Share
0,Magic Johnson,1989,664.5,0.782
1,Michael Jordan,1989,598.8,0.704
2,Karl Malone,1989,362.0,0.426
3,Patrick Ewing,1989,200.0,0.235
4,Hakeem Olajuwon,1989,179.3,0.211


In [32]:
players = pd.read_csv("player.csv")
del players["Unnamed: 0"]
players.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1,Michael Jordan,25,CHI,SG,81,81,40.2,11.9,22.2,...,6.2,8.0,8.0,2.9,0.8,3.6,3.0,32.5,"MVP-2,DPOY-5,AS,NBA1",1989
1,2,Karl Malone,25,UTA,PF,80,80,39.1,10.1,19.5,...,7.4,10.7,2.7,1.8,0.9,3.6,3.6,29.1,"MVP-3,AS,NBA1",1989
2,3,Dale Ellis,28,SEA,SG,82,82,38.9,10.5,20.9,...,2.3,4.2,2.0,1.3,0.3,2.7,2.4,27.5,"AS,NBA3",1989
3,4,Clyde Drexler,26,POR,SG,78,78,39.3,10.6,21.4,...,4.2,7.9,5.8,2.7,0.7,3.2,3.4,27.2,AS,1989
4,5,Alex English,35,DEN,SF,82,82,36.5,11.3,22.9,...,2.2,4.0,4.7,0.8,0.1,2.4,2.1,26.5,AS,1989


In [33]:
# function for people who have multiple teams in one year
def grouper(df):
    if df.shape[0] == 1:
        return df
    else:
        # getting their total scores but listing them as their last team so we still get the team stats with no issue
        row = df[df["Team"] == "TOT"]
        row["Team"] = df.iloc[-1,:]["Team"]
        return row
players = players.groupby(["Player", "Year"]).apply(grouper)

C:\Users\memea\AppData\Local\Temp\ipykernel_47192\2927824250.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  players = players.groupby(["Player", "Year"]).apply(grouper)


In [34]:
players.index = players.index.droplevel()
players.index = players.index.droplevel()
players.head(20)

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
105,94,A.C. Green,25,LAL,PF,82,82,30.6,4.9,9.2,...,5.9,9.0,1.3,1.1,0.7,1.5,2.1,13.3,NaN,1989
560,103,A.C. Green,26,LAL,PF,82,82,33.0,4.7,9.8,...,5.5,8.7,1.1,0.8,0.6,1.4,2.5,12.9,AS,1990
1108,155,A.C. Green,27,LAL,PF,82,21,26.4,3.1,6.6,...,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,NaN,1991
1476,94,A.C. Green,28,LAL,PF,82,53,35.4,4.7,9.8,...,5.6,9.3,1.4,1.1,0.4,1.4,1.7,13.6,NaN,1992
1961,104,A.C. Green,29,LAL,PF,82,55,34.4,4.6,8.6,...,5.2,8.7,1.4,1.1,0.5,1.4,1.8,12.8,NaN,1993
2391,66,A.C. Green,30,PHO,PF,82,55,34.5,5.7,11.3,...,5.8,9.2,1.7,0.9,0.5,1.2,1.7,14.7,NaN,1994
2947,117,A.C. Green,31,PHO,SF,82,52,32.8,3.8,7.5,...,5.8,8.2,1.5,0.7,0.4,1.4,1.8,11.2,NaN,1995
3527,202,A.C. Green,32,PHO,SF,82,36,25.8,2.6,5.4,...,4.7,6.8,0.9,0.5,0.3,1.0,1.7,7.5,NaN,1996
4685,203,A.C. Green,34,DAL,PF,82,68,32.3,3.0,6.5,...,5.5,8.1,1.5,1.0,0.3,0.8,1.9,7.3,NaN,1998
5303,261,A.C. Green,35,DAL,PF,50,35,18.5,2.2,5.1,...,2.9,4.6,0.5,0.6,0.2,0.4,1.4,4.9,NaN,1999


In [35]:
mvpplayers = players.merge(mvps, how="outer", on=["Player", "Year"])
mvpplayers.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Awards,Year,Pts Won,Share
0,94,A.C. Green,25,LAL,PF,82,82,30.6,4.9,9.2,...,1.3,1.1,0.7,1.5,2.1,13.3,NaN,1989,NaN,NaN
1,103,A.C. Green,26,LAL,PF,82,82,33.0,4.7,9.8,...,1.1,0.8,0.6,1.4,2.5,12.9,AS,1990,NaN,NaN
2,155,A.C. Green,27,LAL,PF,82,21,26.4,3.1,6.6,...,0.9,0.7,0.3,1.2,1.4,9.1,NaN,1991,NaN,NaN
3,94,A.C. Green,28,LAL,PF,82,53,35.4,4.7,9.8,...,1.4,1.1,0.4,1.4,1.7,13.6,NaN,1992,NaN,NaN
4,104,A.C. Green,29,LAL,PF,82,55,34.4,4.6,8.6,...,1.4,1.1,0.5,1.4,1.8,12.8,NaN,1993,NaN,NaN


In [36]:
mvpplayers[["Pts Won", "Share"]] = mvpplayers[["Pts Won", "Share"]].fillna(0)
mvpplayers.head()

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Awards,Year,Pts Won,Share
0,94,A.C. Green,25,LAL,PF,82,82,30.6,4.9,9.2,...,1.3,1.1,0.7,1.5,2.1,13.3,NaN,1989,0.0,0.0
1,103,A.C. Green,26,LAL,PF,82,82,33.0,4.7,9.8,...,1.1,0.8,0.6,1.4,2.5,12.9,AS,1990,0.0,0.0
2,155,A.C. Green,27,LAL,PF,82,21,26.4,3.1,6.6,...,0.9,0.7,0.3,1.2,1.4,9.1,NaN,1991,0.0,0.0
3,94,A.C. Green,28,LAL,PF,82,53,35.4,4.7,9.8,...,1.4,1.1,0.4,1.4,1.7,13.6,NaN,1992,0.0,0.0
4,104,A.C. Green,29,LAL,PF,82,55,34.4,4.6,8.6,...,1.4,1.1,0.5,1.4,1.8,12.8,NaN,1993,0.0,0.0


In [37]:
teams = pd.read_csv("team.csv")
del teams["Unnamed: 0"]
teams.head(10)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,52,30,.634,—,116.7,112.9,3.62,1989,New York Knicks*
1,46,36,.561,6.0,111.9,110.4,1.68,1989,Philadelphia 76ers*
2,42,40,.512,10.0,109.2,108.1,1.26,1989,Boston Celtics*
3,40,42,.488,12.0,108.3,110.4,-1.77,1989,Washington Bullets
4,26,56,.317,26.0,103.7,110.1,-5.69,1989,New Jersey Nets
5,20,62,.244,32.0,104.5,113.0,-7.74,1989,Charlotte Hornets
6,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,1989,Central Division
7,63,19,.768,—,106.6,100.8,6.24,1989,Detroit Pistons*
8,57,25,.695,6.0,108.8,101.2,7.95,1989,Cleveland Cavaliers*
9,52,30,.634,11.0,111.0,106.1,5.26,1989,Atlanta Hawks*


In [38]:
teams = teams[~teams["W"].str.contains("Division")]
teams.head(10)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,52,30,.634,—,116.7,112.9,3.62,1989,New York Knicks*
1,46,36,.561,6.0,111.9,110.4,1.68,1989,Philadelphia 76ers*
2,42,40,.512,10.0,109.2,108.1,1.26,1989,Boston Celtics*
3,40,42,.488,12.0,108.3,110.4,-1.77,1989,Washington Bullets
4,26,56,.317,26.0,103.7,110.1,-5.69,1989,New Jersey Nets
5,20,62,.244,32.0,104.5,113.0,-7.74,1989,Charlotte Hornets
7,63,19,.768,—,106.6,100.8,6.24,1989,Detroit Pistons*
8,57,25,.695,6.0,108.8,101.2,7.95,1989,Cleveland Cavaliers*
9,52,30,.634,11.0,111.0,106.1,5.26,1989,Atlanta Hawks*
10,49,33,.598,14.0,108.9,105.3,4.11,1989,Milwaukee Bucks*


In [39]:
teams["Team"] = teams["Team"].str.replace("*", "", regex=False)
teams.head(10)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,52,30,.634,—,116.7,112.9,3.62,1989,New York Knicks
1,46,36,.561,6.0,111.9,110.4,1.68,1989,Philadelphia 76ers
2,42,40,.512,10.0,109.2,108.1,1.26,1989,Boston Celtics
3,40,42,.488,12.0,108.3,110.4,-1.77,1989,Washington Bullets
4,26,56,.317,26.0,103.7,110.1,-5.69,1989,New Jersey Nets
5,20,62,.244,32.0,104.5,113.0,-7.74,1989,Charlotte Hornets
7,63,19,.768,—,106.6,100.8,6.24,1989,Detroit Pistons
8,57,25,.695,6.0,108.8,101.2,7.95,1989,Cleveland Cavaliers
9,52,30,.634,11.0,111.0,106.1,5.26,1989,Atlanta Hawks
10,49,33,.598,14.0,108.9,105.3,4.11,1989,Milwaukee Bucks


In [40]:
teams["Team"].unique()

array(['New York Knicks', 'Philadelphia 76ers', 'Boston Celtics',
       'Washington Bullets', 'New Jersey Nets', 'Charlotte Hornets',
       'Detroit Pistons', 'Cleveland Cavaliers', 'Atlanta Hawks',
       'Milwaukee Bucks', 'Chicago Bulls', 'Indiana Pacers', 'Utah Jazz',
       'Houston Rockets', 'Denver Nuggets', 'Dallas Mavericks',
       'San Antonio Spurs', 'Miami Heat', 'Los Angeles Lakers',
       'Phoenix Suns', 'Seattle SuperSonics', 'Golden State Warriors',
       'Portland Trail Blazers', 'Sacramento Kings',
       'Los Angeles Clippers', 'Orlando Magic', 'Minnesota Timberwolves',
       'Toronto Raptors', 'Vancouver Grizzlies', 'Washington Wizards',
       'Memphis Grizzlies', 'New Orleans Hornets', 'Charlotte Bobcats',
       'New Orleans/Oklahoma City Hornets', 'Oklahoma City Thunder',
       'Brooklyn Nets', 'New Orleans Pelicans'], dtype=object)

In [41]:
# got this straight from a tutorial sorry
abbr = {}

with open("nicknames.csv") as file:
    lines = file.readlines()
for line in lines[1:]:
    abbrev,name = line.replace("\n","").split(",")
    abbr[abbrev] = name
mvpplayers["TeamFull"] = mvpplayers["Team"].map(abbr)
mvpplayers.head(10)

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,Awards,Year,Pts Won,Share,TeamFull
0,94,A.C. Green,25,LAL,PF,82,82,30.6,4.9,9.2,...,1.1,0.7,1.5,2.1,13.3,NaN,1989,0.0,0.0,Los Angeles Lakers
1,103,A.C. Green,26,LAL,PF,82,82,33.0,4.7,9.8,...,0.8,0.6,1.4,2.5,12.9,AS,1990,0.0,0.0,Los Angeles Lakers
2,155,A.C. Green,27,LAL,PF,82,21,26.4,3.1,6.6,...,0.7,0.3,1.2,1.4,9.1,NaN,1991,0.0,0.0,Los Angeles Lakers
3,94,A.C. Green,28,LAL,PF,82,53,35.4,4.7,9.8,...,1.1,0.4,1.4,1.7,13.6,NaN,1992,0.0,0.0,Los Angeles Lakers
4,104,A.C. Green,29,LAL,PF,82,55,34.4,4.6,8.6,...,1.1,0.5,1.4,1.8,12.8,NaN,1993,0.0,0.0,Los Angeles Lakers
5,66,A.C. Green,30,PHO,PF,82,55,34.5,5.7,11.3,...,0.9,0.5,1.2,1.7,14.7,NaN,1994,0.0,0.0,Phoenix Suns
6,117,A.C. Green,31,PHO,SF,82,52,32.8,3.8,7.5,...,0.7,0.4,1.4,1.8,11.2,NaN,1995,0.0,0.0,Phoenix Suns
7,202,A.C. Green,32,PHO,SF,82,36,25.8,2.6,5.4,...,0.5,0.3,1.0,1.7,7.5,NaN,1996,0.0,0.0,Phoenix Suns
8,203,A.C. Green,34,DAL,PF,82,68,32.3,3.0,6.5,...,1.0,0.3,0.8,1.9,7.3,NaN,1998,0.0,0.0,Dallas Mavericks
9,261,A.C. Green,35,DAL,PF,50,35,18.5,2.2,5.1,...,0.6,0.2,0.4,1.4,4.9,NaN,1999,0.0,0.0,Dallas Mavericks


In [42]:
teams = teams.rename(columns = {"Team": "TeamFull"})
stats = mvpplayers.merge(teams, how="outer", on=["TeamFull", "Year"])
stats.head(10)

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,Pts Won,Share,TeamFull,W,L,W/L%,GB,PS/G,PA/G,SRS
0,170,Antoine Carr,27,ATL,PF,78,12,19.1,2.9,6.0,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
1,143,Cliff Levingston,28,ATL,PF,80,52,27.3,3.8,7.1,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
2,91,Doc Rivers,27,ATL,PG,76,76,32.4,4.9,10.7,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
3,7,Dominique Wilkins,29,ATL,SF,80,80,37.5,10.2,22.0,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
4,300,Duane Ferrell,23,ATL,SF,41,0,5.6,0.9,2.0,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
5,317,Dudley Bradley,31,ATL,SG,38,0,7.0,0.7,2.3,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
6,139,John Battle,26,ATL,SG,82,0,20.4,3.5,7.7,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
7,237,Jon Koncak,25,ATL,C,74,22,20.7,1.9,3.6,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
8,24,Moses Malone,33,ATL,C,81,80,35.5,6.6,13.5,...,3.0,0.004,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26
9,315,Ray Tolbert,30,ATL,PF,50,0,6.8,0.8,1.9,...,0.0,0.000,Atlanta Hawks,52,30,.634,11.0,111.0,106.1,5.26


In [43]:
stats["GB"] = stats["GB"].str.replace("—", "0")
stats = stats.apply(pd.to_numeric, errors="ignore")
stats.dtypes

C:\Users\memea\AppData\Local\Temp\ipykernel_47192\2309053071.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  stats = stats.apply(pd.to_numeric, errors="ignore")


Rk          float64
Player       object
Age         float64
Team         object
Pos          object
G           float64
GS          float64
MP          float64
FG          float64
FGA         float64
FG%         float64
3P          float64
3PA         float64
3P%         float64
2P          float64
2PA         float64
2P%         float64
eFG%        float64
FT          float64
FTA         float64
FT%         float64
ORB         float64
DRB         float64
TRB         float64
AST         float64
STL         float64
BLK         float64
TOV         float64
PF          float64
PTS         float64
Awards       object
Year          int64
Pts Won     float64
Share       float64
TeamFull     object
W           float64
L           float64
W/L%        float64
GB          float64
PS/G        float64
PA/G        float64
SRS         float64
dtype: object

In [44]:
stats.to_csv("fullstats.csv")